# 도발 사건 데이터 정제 및 CSV 통합 저장
작성자: 김지민, 김영성

이 노트북은 북한의 주요 도발 사건 데이터를 **유형별로 정제**하고,  
최종적으로 **하나의 통합된 CSV 파일(`provocation_events.csv`)**로 저장하는 과정을 다룹니다.

## 주요 내용
- 도발 유형: 미사일 / 핵실험 / 오물풍선 / 피습 사건
- 각 데이터셋에서 불필요한 열 제거 및 컬럼명 정리
- 날짜 형식 통일 (`pd.to_datetime` 적용)
- 북한/남한 정권 구분 컬럼 추가 (`n_gov`, `s_gov`)
- 최종 통합 데이터프레임 생성 및 저장

## 저장 포맷
- 컬럼: `Type`, `Date`, `Case`, `Latitude`, `Longitude`, `Year`, `n_gov`, `s_gov`
- 저장 경로: `data/provocation_events.csv`
---

In [1]:
import re
import pandas as pd

----------------

## 공통 전처리 작업
- 불필요한 열 제거
- 도발 유형 열(`Type`) 추가 및 컬럼명 표준화
- 날짜 형식 통일 (`pd.to_datetime`)
- 저장 형식: `[Type, Date, Case, Latitude, Longtitude]`

## 1. 미사일

In [2]:
df_missile = pd.read_excel('data/도발유형/north_korea_missile_test_database.xlsx')
df_missile.head()

,Unnamed: 0,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,Unnamed: 10,Unnamed: 11,Unnamed: 12,Unnamed: 13,Unnamed: 14,Unnamed: 15,Unnamed: 16,Unnamed: 17,Unnamed: 18
0,F1,Date,Date Entered/Updated,Launch Time (UTC),Missile Name,Missile Type,Launch Agency/Authority,Facility Name,Facility Location,Other Name,Facility Latitude,Facility Longitude,Landing Location,Apogee,Distance Travelled,Confirmation Status,Test Outcome,Additional Information,Source(s)
1,1,1984-04-09 00:00:00,2016-12-23 00:00:00,NaN,Scud-B,SRBM,NaN,Tonghae Satellite Launching Ground,"Hwadae County, North Hamgyong Province",Musudan-ri,40.849997,129.666664,Unknown,Unknown,Unknown,Confirmed,Success,First known test by North Korea of a missile w...,http://www.nti.org/media/pdfs/north_korea_miss...
2,2,1984-04-09 00:00:00,2016-12-23 00:00:00,NaN,Scud-B,SRBM,NaN,Tonghae Satellite Launching Ground,"Hwadae County, North Hamgyong Province",Musudan-ri,40.849997,129.666664,Unknown,Unknown,Unknown,Confirmed,Success,NaN,http://www.nti.org/media/pdfs/north_korea_miss...
3,3,1984-04-09 00:00:00,2016-12-23 00:00:00,NaN,Scud-B,SRBM,NaN,Tonghae Satellite Launching Ground,"Hwadae County, North Hamgyong Province",Musudan-ri,40.849997,129.666664,Unknown,200 km,Unknown,Confirmed,Success,NaN,http://www.astronautix.com/g/gitdaeryung.html;...
4,4,1984-09-01 00:00:00,2016-12-23 00:00:00,NaN,Scud-B,SRBM,NaN,Tonghae Satellite Launching Ground,"Hwadae County, North Hamgyong Province",Musudan-ri,40.849997,129.666664,Unknown,Unknown,Unknown,Confirmed,Failure,NaN,"Joseph S. Bermudez, ""A History of Ballistic Mi..."


In [3]:
df_missile = df_missile.drop(['Unnamed: 0', 'Unnamed: 2','Unnamed: 3', 'Unnamed: 5', 
                              'Unnamed: 6', 'Unnamed: 7', 'Unnamed: 8', 'Unnamed: 9', 
                              'Unnamed: 12', 'Unnamed: 13', 'Unnamed: 14', 'Unnamed: 15', 
                              'Unnamed: 16', 'Unnamed: 17', 'Unnamed: 18'], axis=1)
df_missile['Type'] = '미사일'
df_missile.columns = ['Date', 'Case', 'Latitude', 'Longtitude', 'Type']

df_missile = df_missile[['Type', 'Date', 'Case', 'Latitude', 'Longtitude']]
df_missile = df_missile.drop(0)

df_missile['Date'] = pd.to_datetime(df_missile['Date'])
df_missile.to_csv('data/missile.csv', index=False)
df_missile.head()

,Type,Date,Case,Latitude,Longtitude
1,미사일,1984-04-09,Scud-B,40.849997,129.666664
2,미사일,1984-04-09,Scud-B,40.849997,129.666664
3,미사일,1984-04-09,Scud-B,40.849997,129.666664
4,미사일,1984-09-01,Scud-B,40.849997,129.666664
5,미사일,1984-09-01,Scud-B,40.849997,129.666664


-----------

## 2. 핵

In [4]:
df_nuclear = pd.read_excel('data/도발유형/north_korea_nuclear_tests.xlsx')
df_nuclear.head()

,Date,Type,Event,Description,Resources,Latitude,Longtitude
0,2006-10-09,Nuclear Provocation,Nuclear Test,풍계리 핵실험장에서 시행된 북한의 1차 핵실험,http://georgewbush-whitehouse.archives.gov/new...,41.28505,129.10840
1,2009-05-25,Nuclear Provocation,Nuclear Test / Short-range missile launch,풍계리 핵실험장에서 시행된 북한의 2차 핵실험,https://www.whitehouse.gov/the-press-office/st...,41.29142,129.08167
2,2013-02-12,Nuclear Provocation,Nuclear Test,풍계리 핵실험장에서 시행된 북한의 3차 핵실험,http://www.wsj.com/articles/SB1000142412788732...,41.26809,129.08076
3,2016-01-06,Nuclear Provocation,Nuclear Test,풍계리 핵실험장에서 시행된 북한의 4차 핵실험,http://www.reuters.com/article/us-northkorea-n...,41.30900,129.03399
4,2016-09-09,Nuclear Provocation,Nuclear Test,풍계리 핵실험장에서 시행된 북한의 5차 핵실험,http://www.cnn.com/2016/09/08/asia/north-korea...,41.29800,129.01500


In [5]:
df_nuclear = df_nuclear.drop(['Type', 'Event', 'Resources'], axis=1)

df_nuclear['Type'] = '핵실험'
df_nuclear.columns = ['Date', 'Case', 'Latitude', 'Longtitude', 'Type']
df_nuclear = df_nuclear[['Type', 'Date', 'Case', 'Latitude', 'Longtitude']]

df_nuclear['Date'] = pd.to_datetime(df_nuclear['Date'])
df_nuclear.to_csv('data/nuclear.csv', index=False)
df_nuclear.head()

,Type,Date,Case,Latitude,Longtitude
0,핵실험,2006-10-09,풍계리 핵실험장에서 시행된 북한의 1차 핵실험,41.28505,129.10840
1,핵실험,2009-05-25,풍계리 핵실험장에서 시행된 북한의 2차 핵실험,41.29142,129.08167
2,핵실험,2013-02-12,풍계리 핵실험장에서 시행된 북한의 3차 핵실험,41.26809,129.08076
3,핵실험,2016-01-06,풍계리 핵실험장에서 시행된 북한의 4차 핵실험,41.30900,129.03399
4,핵실험,2016-09-09,풍계리 핵실험장에서 시행된 북한의 5차 핵실험,41.29800,129.01500


--------

## 3. 오물풍선

In [6]:
df_balloon = pd.read_csv('data/도발유형/오물풍선.csv')
df_balloon.head()

,Wave number,Location,LAT,LON,Province,Korean Name,Date,Time,Source
0,First Wave,"Duma-myeon, Gyeryong-si, Chungcheongnam-do",36.254268,127.260809,Chungcheongnam Province,충남 계룡시 두마면,"May 29, 2024",3:05 AM,Donga Ilbo
1,First Wave,"Tanhyeon-myeon, Paju-si",37.809190,126.710150,Gyeonggi Province,파주시 탄현면,"May 29, 2024",5:15 AM,Newsis
2,First Wave,"Wicheon-myeon, Geochang-gun, Gyeongsangnam-do",35.750860,127.832670,Gyeongsangnam Province,경남 거창군 위천면,"May 29, 2024",5:30 AM,Donga Ilbo
3,First Wave,"Mujueup, Muju-gun, Jeollabuk-do",35.931720,127.711180,Jeollabuk Province,전북자치도 무주군 무주읍,"May 29, 2024",5:45 AM,https://www.news1.kr/articles/?5430632
4,First Wave,"Seongnam-si, Sujeong-gu",37.450339,127.146293,Gyeonggi Province,성남시 수정구,"May 29, 2024",5:49 AM,https://mobile.newsis.com/view.html?ar_id=NISX...


In [7]:
df_balloon = df_balloon.drop(['Location', 'Province', 'Korean Name', 'Time', 'Source'], axis=1)

df_balloon['Type'] = '오물풍선'
df_balloon.columns = ['Case', 'Latitude', 'Longtitude', 'Date', 'Type']
df_balloon = df_balloon[['Type', 'Date', 'Case', 'Latitude', 'Longtitude']]

df_balloon['Date'] = pd.to_datetime(df_balloon['Date'])
df_balloon.to_csv('data/balloon.csv', index=False)
df_balloon.head()

,Type,Date,Case,Latitude,Longtitude
0,오물풍선,2024-05-29,First Wave,36.254268,127.260809
1,오물풍선,2024-05-29,First Wave,37.809190,126.710150
2,오물풍선,2024-05-29,First Wave,35.750860,127.832670
3,오물풍선,2024-05-29,First Wave,35.931720,127.711180
4,오물풍선,2024-05-29,First Wave,37.450339,127.146293


-------

## 4. 피습

## 1) 피습사건1 (1945~1980년대)

In [8]:
df_attack1 = pd.read_excel('data/도발유형/1945~1980년대_국회도서관.xlsx')
df_attack1.head()

,날짜,사건명,사건장소,위도,경도,내용
0,1945. 9. 5.,"조선정판사, 위조지폐 사건",NaN,NaN,NaN,"이만출, 권오지, 박탁용 등이 경제적인 혼란을 일으키기 위해 5회에 걸쳐 1,200..."
1,1946. 1. 18.,학병동맹사건,NaN,NaN,NaN,신탁통치 반대시위를 하던 학생들에게 좌익학병동맹이 총기를 난사하여 유혈충돌을 일으켰다.
2,1946. 1. 29.,정계요인 암살 및 경찰서 방화기도 사건,NaN,NaN,NaN,한민당의 주석홍무익 송진우를 암살하려다 실패하고 구속된 공산당원 김길홍을 구출하기 ...
3,1946. 10. 1.,대구폭동 사건,대구,35.871567,128.601259,"공산당의 선동하에 대구를 중심으로 한 경북, 우도도지에서 폭동이 발생하여, 피살자 ..."
4,1946. 10. 16.,조병옥 경무부장 암살미수 사건,NaN,NaN,NaN,"공산당 산하인 민청 소속의 유옥렬, 이충락 등이 조병옥에게 수류탄을 던졌으나 불발에..."


In [9]:
df_attack1 = df_attack1.drop(['사건장소', '내용'], axis=1)
df_attack1['Type'] = '피습 사건'
df_attack1.columns = ['Date', 'Case', 'Latitude', 'Longtitude', 'Type']
df_attack1 = df_attack1[['Type', 'Date', 'Case', 'Latitude', 'Longtitude']]

* 연도.월.일 → 연도. 월. 일 (띄어쓰기 추가)
* 연도.월. → 연도. 월. 1. (일 누락 시 1일로 보정)
* 날짜 형식 변환

In [10]:
df_attack1['Date'] = df_attack1['Date'].str.replace(r'(\d{4})\.(\d{1,2})\.(\d{1,2})\.', r'\1. \2. \3.', regex=True)
df_attack1['Date'] = df_attack1['Date'].str.replace(r'(\d{4})\.(\d{1,2})\.$', r'\1. \2. 1.', regex=True)
df_attack1['Date'] = pd.to_datetime(df_attack1['Date'], format='%Y. %m. %d.', errors='coerce')

In [11]:
df_attack1.head()

,Type,Date,Case,Latitude,Longtitude
0,피습 사건,1945-09-05,"조선정판사, 위조지폐 사건",NaN,NaN
1,피습 사건,1946-01-18,학병동맹사건,NaN,NaN
2,피습 사건,1946-01-29,정계요인 암살 및 경찰서 방화기도 사건,NaN,NaN
3,피습 사건,1946-10-01,대구폭동 사건,35.871567,128.601259
4,피습 사건,1946-10-16,조병옥 경무부장 암살미수 사건,NaN,NaN


##  2) 피습사건2(1990~2010년대)

In [12]:
df_attack2 = pd.read_excel('data/도발유형/1990~2010년대_국가정보원.xlsx')
df_attack2.head()

,날짜,사건명,사건장소,위도,경도,내용
0,1983.10.09,미얀마 아웅산묘소 폭탄테러,미얀마,21.710000,95.970000,"북한공작원이 원격조종 폭탄 설치, 17명 순국"
1,1984.09.24,북한 무장간첩 민간인 무차별 살상,대구,35.871567,128.601259,"민간인 2명 사망, 1명 부상"
2,1983.12.03,부산 다대표 무장간첩 침투,부산,35.179926,129.075095,NaN
3,1986.09.14,김포공항 청사 폭탄테러,김포공항,37.571370,126.802960,"폭발물 설치(사망 5명, 중경상 32명)"
4,1987.01.15,제27 동진호 조업 중 납북,백령도,37.960342,124.674943,북한군이 어민 납북


In [13]:
df_attacks2 = df_attack2.drop(['사건장소', '내용'], axis=1)
df_attacks2['Type'] = '피습 사건'
df_attacks2.columns = ['Date', 'Case', 'Latitude', 'Longtitude', 'Type']
df_attack2 = df_attacks2[['Type', 'Date', 'Case', 'Latitude', 'Longtitude']]

* 연도.월.일 → 연도. 월. 일 (띄어쓰기 추가)
* 연도.월. → 연도. 월. 1. (일 누락 시 1일로 보정)
* 날짜 형식 변환

In [14]:
df_attack2['Date'] = df_attack2['Date'].astype(str)
mask = df_attack2['Date'].str.fullmatch(r'\d{4}\.\d{1,2}\.')
df_attack2.loc[mask, 'Date'] = df_attack2.loc[mask, 'Date'] + '1'
df_attack2['Date'] = pd.to_datetime(df_attack2['Date'], errors='coerce')

df_attack2.head()

,Type,Date,Case,Latitude,Longtitude
0,피습 사건,1983-10-09,미얀마 아웅산묘소 폭탄테러,21.710000,95.970000
1,피습 사건,1984-09-24,북한 무장간첩 민간인 무차별 살상,35.871567,128.601259
2,피습 사건,1983-12-03,부산 다대표 무장간첩 침투,35.179926,129.075095
3,피습 사건,1986-09-14,김포공항 청사 폭탄테러,37.571370,126.802960
4,피습 사건,1987-01-15,제27 동진호 조업 중 납북,37.960342,124.674943


## 3) 피습사건1, 피습사건2 병합

In [15]:
df_attack = df_attack1.merge(df_attack2, how='outer')
df_attack.to_csv('data/attack.csv', index=False)

----

## 5. 정권 구분 및 도발 유형 통합

- 연도 칼럼 생성

In [16]:
for df in [df_attack, df_balloon, df_missile, df_nuclear]:
    df['Year'] = pd.to_datetime(df['Date'], errors='coerce').dt.year

- 북한 정권 구분 함수

In [17]:
def north_korea_regime(year):
    if year < 1948:
        return '임시정부'
    elif year < 1994:
        return '김일성'
    elif year < 2011:
        return '김정일'
    else:
        return '김정은'

- 남한 정권 구분 함수

In [18]:
def south_korea_regime(year):
    if year < 1948:
        return '임시정부'
    elif year < 1960:
        return '이승만'
    elif year < 1962:
        return '윤보선'
    elif year < 1979:
        return '박정희'
    elif year < 1980:
        return '최규하'
    elif year < 1988:
        return '전두환'
    elif year < 1993:
        return '노태우'
    elif year < 1998:
        return '김영삼'
    elif year < 2003:
        return '김대중'
    elif year < 2008:
        return '노무현'
    elif year < 2013:
        return '이명박'
    elif year < 2017:
        return '박근혜'
    elif year < 2022:
        return '문재인'
    else:
        return '윤석열'

- 북한/남한 정권 컬럼 추가

In [19]:
for df in [df_attack, df_balloon, df_missile, df_nuclear]:
    df['n_gov'] = df['Year'].apply(north_korea_regime)
    df['s_gov'] = df['Year'].apply(south_korea_regime)

- 도발 유형별 전처리된 데이터 프레임을 하나로 합쳐 통합 분석용 데이터 생성

In [20]:
provocation_events = pd.concat([df_attack, df_balloon, df_missile, df_nuclear], ignore_index=True)
provocation_events.to_csv('data/provocation_events.csv', index=False)

------